# Function Definition

In [2]:
from scipy import optimize
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
from sympy import symbols, diff
import sympy as sp

In [3]:
def F(m, phi_i, d = 1*10**(-3), lam = 650*10**(-9)):
    """"
    This function takes as inputs the parameters from our experiment and returns another function, only in terms
    of n, such that it can later be optimized while taking our parameteres as fixed.
    
    Parameters
    -------
    m : the number of fringes counter
    d : thickness of the material plate
    lam : wavelength of the light source
    phi_i : angle through which we rotate the plate, to the normal angle of mirror
    
    Returns
    -------
    funk_to_optimize : a function containing the parameters as fixed, in terms of refractive index 'n'
    
    Notes
    -------
    
    """
    
    def funk_to_optimize(n):
        
        phi_r = math.asin(math.sin(phi_i)/n)
        
        return m - (2*d)/lam * (n*(1/math.cos(phi_r) - 1)
                                + 1 - (math.cos(phi_i - phi_r)/math.cos(phi_r)))
    
    return funk_to_optimize

In [4]:
def refrakt(input_data, x0 = 1.4, x1 = 1.5):
    """
    This one takes a list of data and returns the refractive index
    
    Params
    -------
    input_data : experimental data that contains number of fringes 
                 and angle of rotation
    x0 = initial guess for optimization algorithm
    x1 = second guess for optimization algorithm
    
    Returns
    -------
    output_data : list of refractive indices
    
    Example
    -------
    >>>refrakt([[20, 10.4], [40, 14.7]])
    [4.79289677e+34, 4.28057195e+31]
    
    Notes
    -------
    We know that we are measuring glass-like material, so we expect 
    the refractive index to be somewhere between 1.4 and 1.5, hence 
    the default values for the initial two guesses
    
    """
    output_data = []
    
    for arr in input_data:
        
        m = arr[0]
        phi_i = arr[1]
    
        func = F(m, phi_i)
        n = optimize.root_scalar(func, x0 = x0, x1 = x1)
    
        output_data.append(n.root)
        
    return output_data

# Data Import and Analysis

In [5]:
def load(url):
    run = pd.read_csv(url, dtype = "float").to_numpy()
    
    run[:,1] = np.radians(run[:,1])
    
    return run

In [6]:
runB1 = load("https://raw.githubusercontent.com/diodeamy/Waves-And-Optics/main/data/data_BR1.csv")
runB2 = load("https://raw.githubusercontent.com/diodeamy/Waves-And-Optics/main/data/data_BR2.csv")
runB3 = load("https://raw.githubusercontent.com/diodeamy/Waves-And-Optics/main/data/data_BR3.csv")
runW1 = load("https://raw.githubusercontent.com/diodeamy/Waves-And-Optics/main/data/data_WR1.csv")
runW2 = load("https://raw.githubusercontent.com/diodeamy/Waves-And-Optics/main/data/data_WR2.csv")

In [7]:
#here we wish to make a function that returns the refractive indeces for each pair of angle-fringe 
#we also wish for it to compute y and x values to be plotted for the error function (this will need to take
#the errors of angle and lalal into account)

In [8]:
def matias(run):
    
    lam = 650*10**(-9)
    d = 1*10**3
    phi_i = run[:,1]
    m = run[:,0]
    
    n = np.array(refrakt(run))
    
    phi_r = np.arcsin(np.sin(phi_i/n))
    
    y = (lam/2*d)*m - 1 + np.cos(phi_i - phi_r)/np.cos(phi_r)
    
    x = 1/np.cos(phi_r) - 1
    
    return (x, y)

In [9]:
x, y = matias(runB1)

/Users/ruxi/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/zeros.py:341: RuntimeWarning: Tolerance of 7.590522150624823e+24 reached.
  warnings.warn(msg, RuntimeWarning)
/Users/ruxi/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/zeros.py:341: RuntimeWarning: Tolerance of 1.1113529697057186e+24 reached.
  warnings.warn(msg, RuntimeWarning)


In [22]:
def slope_and_error(run):
    """
    this one gives the slope and the error in the slope for a linear regression (doesn't account for errors)
    
    Params
    -------
    run: the data
    
    Returns
    -------
    "slope": slope given by regression
    "error": standard error
    """
    result = linregress(matias(run))
    
    return {"slope": result.slope, "error": result.stderr}

# The one where I create error functions

In [20]:
#we need to let the computer know that we want it to differentiate with respect to certain variables

##our errors are in phi_i, m, lam, d and the n we used in the trig functions

def y_partials():
    """
    this one takes partial derivatives of our function for y (NOT CALLABLE)
    
    Params
    -------
    none
    
    Returns
    -------
    list of dictionaries with the symbols(the varialbes you need differentiation wrt) as keys
    and with differentials of x wrt each variable (phi_i, lam, d, m, n) as values
    """
    
    phi_i, m, lam, d, n = symbols('phi_i m lam d n', real = True)

    phi_r = sp.asin(sp.sin(phi_i/n))

    y = (lam/2*d)*m - 1 + sp.cos(phi_i - phi_r)/sp.cos(phi_r)
    
    diff_y_phi_i = diff(y, phi_i)
    diff_y_lam = diff(y, lam)
    diff_y_d = diff(y, d)
    diff_y_m = diff(y, m)
    diff_y_n = diff(y, n)
    
    return [
        {"symbol": phi_i, "derivative": diff_y_phi_i},
        {"symbol": lam, "derivative": diff_y_lam},
        {"symbol": d, "derivative": diff_y_d},
        {"symbol": m, "derivative": diff_y_m},
        {"symbol": n, "derivative": diff_y_n}
    ]
     

In [21]:
def x_partials():
    """
    this one takes partial derivatives of our function for x (NOT CALLABLE)
    
    Params
    -------
    none
    
    Returns
    -------
    list of dictionaries with the symbols(the varialbes you need differentiation wrt) as keys
    and with differentials of x wrt each variable (phi_i, n) as values
    
    """
    
    phi_i, n = symbols('phi_i n', real = True)
    
    phi_r = sp.asin(sp.sin(phi_i/n))
    x = 1/sp.cos(phi_r) - 1
    
    diff_x_n = diff(x, n)
    diff_x_phi_i = diff(x, phi_i)
    
    return [
        {"symbol": phi_i, "derivative": diff_x_phi_i}, 
        {"symbol": n, "derivative": diff_x_n}
    ]

In [29]:
def x_errors(run):
    """
    This function finds errors for the x values(given by matias)
    
    Paramteres
    -------
    delta_phi : error in the angle, it is fixed at 0.2deg or 0.003490658503988659 radians
    delta_n : error in the refractive index, given by the slope_and_error (uses scipy.stats.linregress)
    run : the data
    
    Returns
    -------
    x_del : error in x
    """
    
    #gather the data from the y_partials function (symbols and derivatives separately)
    x_partials_results = x_partials()
    
    phi_i = x_partials_results[0]["symbol"]
    x_partial_derivative_wrt_phi_i = x_partials_results[0]["derivative"]
    
    n = x_partials_results[1]["symbol"]
    x_partial_derivative_wrt_n = x_partials_results[1]["derivative"]

    #make the derivative a callable function
    x_partial_derivative_callable_phi_i = sp.lambdify([phi_i, n], x_partial_derivative_wrt_phi_i)
    x_partial_derivative_callable_n = sp.lambdify([phi_i, n], x_partial_derivative_wrt_n)
    
    #use the points from the run for each error
    phi_is = np.array(run[:,1])
    ns = np.array(refrakt(run))
    
    #declare errors
    delta_phi = (0.2*(math.pi)/180)
    delta_n = slope_and_error(run)["slope"]

    #final exression for error
    a = (x_partial_derivative_callable_phi_i(phi_is, ns) * delta_phi)**2
    b = (x_partial_derivative_callable_n(phi_is, ns) * delta_n)**2
    
    x_del = np.sqrt(a + b)
    
    return x_del

0.003490658503988659

In [40]:
x_errors(runB1)

array([4.22962065e-54, 3.94612363e-52, 7.49333688e-07, 2.77874501e-04,
       1.36224987e-03, 2.45957816e-03, 4.15286495e-03, 6.52942132e-03,
       9.66071524e-03, 1.23203596e-02, 1.56928225e-02, 1.98207135e-02,
       2.47505866e-02, 2.76833251e-02, 3.13989239e-02, 3.58974282e-02,
       4.26157091e-02, 4.87801386e-02, 5.58271821e-02, 6.07907848e-02,
       6.66449742e-02, 7.49102088e-02, 8.11027320e-02, 8.52386633e-02,
       9.03506380e-02, 9.49666977e-02, 9.91214234e-02, 1.05687303e-01,
       1.11801891e-01, 1.17483550e-01])

In [46]:
def y_errors(run):
    """
    this function give the errors in y for the y values (given by matias)
    
    """
    
    #gather the data from the y_partials function (symbols and derivatives separately)
    y_partials_result = y_partials()
    
    phi_i = y_partials_result[0]["symbol"]
    y_partial_derivative_wrt_phi_i = y_partials_result[0]["derivative"]
    
    lam = y_partials_result[1]["symbol"]
    y_partial_derivative_wrt_lam = y_partials_result[1]["derivative"]
    
    d = y_partials_result[2]["symbol"]
    y_partial_derivative_wrt_d = y_partials_result[2]["derivative"]
    
    m = y_partials_result[3]["symbol"]
    y_partial_derivative_wrt_m = y_partials_result[3]["derivative"]
    
    n = y_partials_result[4]["symbol"]
    y_partial_derivative_wrt_n = y_partials_result[4]["derivative"]
    
    #make the derivative a callable function
    y_partial_derivative_callable_phi_i = sp.lambdify([phi_i, lam, d, m, n], y_partial_derivative_wrt_phi_i)
    y_partial_derivative_callable_lam = sp.lambdify([phi_i, lam, d, m, n], y_partial_derivative_wrt_lam)
    y_partial_derivative_callable_d = sp.lambdify([phi_i, lam, d, m, n], y_partial_derivative_wrt_d)
    y_partial_derivative_callable_m = sp.lambdify([phi_i, lam, d, m, n], y_partial_derivative_wrt_m)
    y_partial_derivative_callable_n = sp.lambdify([phi_i, lam, d, m, n], y_partial_derivative_wrt_n)
    
    #use the points from the run for each error
    phi_i = np.array(run[:,1])
    ms = np.array(run[:,0])
    ns = np.array(refrakt(run))
    
    #declare fixed values
    d = 1 * 10**(-3)
    lam = 650* 10**(-9)
    
    #declare errors
    delta_phi_i = (0.2*(math.pi)/180)
    delta_lam = 1*10**(-9)
    delta_d = 0.1*10**(-3)
    delta_m = 2
    delta_n = slope_and_error(run)["slope"]
    
    
    #final expression for error
    a = (y_partial_derivative_callable_phi_i(phi_i, lam, d, ms, ns) * delta_phi_i)**2
    b = (y_partial_derivative_callable_lam(phi_i, lam, d, ms, ns)* delta_lam)**2
    c = (y_partial_derivative_callable_d(phi_i, lam, d, ms, ns)* delta_d)**2
    d = (y_partial_derivative_callable_m(phi_i, lam, d, ms, ns)* delta_m)**2
    e = (y_partial_derivative_callable_n(phi_i, lam, d, ms, ns)* delta_n)**2
    
    y_del = np.sqrt(a + b + c + d + e)
    
    return y_del

In [47]:
y_errors(runB1)

array([6.09203909e-05, 1.21822225e-04, 1.64272657e-04, 8.33187368e-04,
       2.78331636e-03, 4.66209871e-03, 7.32853755e-03, 1.08358027e-02,
       1.52261967e-02, 1.92130295e-02, 2.40653282e-02, 2.98130834e-02,
       3.64894697e-02, 4.13088424e-02, 4.70485577e-02, 5.37188880e-02,
       6.27648949e-02, 7.13914174e-02, 8.10407229e-02, 8.87716806e-02,
       9.75328516e-02, 1.08854095e-01, 1.18253520e-01, 1.25710046e-01,
       1.34264834e-01, 1.42416893e-01, 1.50180041e-01, 1.60577011e-01,
       1.70611794e-01, 1.80289168e-01])